In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.geometry import LineString
from utils.save_shp import save_shp
from utils.load_csv import load_csv
from utils.load_shp import load_shp

In [2]:
df_posicoes = load_csv('df_posicoes.csv')
distritos = load_shp("distritos.shp")
consumo_diesel = load_csv('consumo_diesel.csv')

In [3]:
def get_tipo_onibus(id_onibus):

    onibus = df_posicoes[df_posicoes['id_onibus']==id_onibus]

    return onibus['modelo'].unique()[0]

def get_fator_consumo(tipo_veiculo):

    cosumo_modelo = consumo_diesel[consumo_diesel['tecnologia']==tipo_veiculo]

    return cosumo_modelo['com_ar_l_km'].unique()[0]

def get_consumo_co2(row, km_rodados):

    fator_emissao_co2 = 2.671 

    consumo_l_km = row["com_ar_l_km"] if row["tem_ar"] else row["sem_ar_l_km"]

    emissao_co2 = km_rodados * consumo_l_km * fator_emissao_co2
    
    return emissao_co2

In [4]:
def get_modelo_line(row):
    return consumo_diesel.loc[consumo_diesel['tecnologia']==row['modelo']]  

def get_consumo_l_km(row):
    consumo_line = get_modelo_line(row)
    consumo_l = consumo_line['com_ar_l_km'] if row['tem_ar'] else consumo_line['sem_ar_l_km']
    consumo_value = row['distancia'] * consumo_l
    return consumo_value

def get_consumo_kg_km(row):
    consumo_line = get_modelo_line(row)
    consumo_kg = consumo_line['com_ar_kg_km'] if row['tem_ar'] else consumo_line['sem_ar_kg_km']
    consumo_value = row['distancia'] * consumo_kg
    return consumo_value

In [5]:
#transformar em gdf + epsg:31983
gdf = gpd.GeoDataFrame(df_posicoes, geometry=gpd.points_from_xy(df_posicoes.lon, df_posicoes.lat), crs="EPSG:4326")

gdf = gdf.to_crs("EPSG:31983")

In [6]:
#dropar pontos únicos (ônibus que só tem 1 ponto)
dropar_filter = (
    gdf['id_onibus']
    .value_counts()
    [(gdf['id_onibus'].value_counts() == 1)]
    .index
)

gdf = gdf[~gdf['id_onibus'].isin(dropar_filter)].reset_index(drop=True)

In [7]:
#Criar linestrings
trajetos = (
    gdf
    .groupby("id_onibus")["geometry"]
    .apply(lambda x: LineString(x.tolist()))
)
trajetos = gpd.GeoDataFrame(
    trajetos, 
    geometry="geometry", 
    crs="EPSG:31983"
).reset_index()


In [8]:
#overlay
gdf_overlay = gpd.overlay(trajetos, distritos, how="intersection")

gdf_overlay['distancia'] = gdf_overlay.geometry.length / 1000 #distancia em km

#gdf_overlay.explore()

In [9]:
dropar_columnns ={
    'cd_identif',
    'cd_identi0',
    'tx_escala',
    'sg_fonte_o',
    'dt_criacao',
    'dt_atualiz',
    'cd_usuario',
    'cd_tipo_di',
    'qt_area_qu',
    'qt_area_me'
}
gdf_overlay.drop(columns=dropar_columnns, inplace=True)
gdf_overlay.shape


(14167, 6)

In [10]:

gdf_overlay.head(3)

,id_onibus,cd_distrit,nm_distrit,sg_distrit,geometry,distancia
0,3106,24,CIDADE LIDER,CLD,"MULTILINESTRING ((345990.634 7394759.92, 34602...",0.066289
1,3117,76,SAPOPEMBA,SAP,"MULTILINESTRING ((347423.675 7389607.4, 347253...",0.223807
2,3117,73,SAO MATEUS,SMT,"MULTILINESTRING ((350506.219 7387400.996, 3474...",10.452956


In [11]:
ids_uniques = df_posicoes['id_onibus'].unique()
for u in ids_uniques:
    gdf_overlay.loc[gdf_overlay['id_onibus'] == u, 'modelo'] = (
        df_posicoes
        .loc[df_posicoes['id_onibus'] == u, 'modelo']
        .values[0]
    )
    gdf_overlay.loc[gdf_overlay['id_onibus'] == u, 'is_eletrico'] = (
        df_posicoes
        .loc[df_posicoes['id_onibus'] == u, 'is_eletrico']
        .values[0]
    )


gdf_overlay.shape

(14167, 8)

In [12]:
gdf_overlay.head(3)

,id_onibus,cd_distrit,nm_distrit,sg_distrit,geometry,distancia,modelo,is_eletrico
0,3106,24,CIDADE LIDER,CLD,"MULTILINESTRING ((345990.634 7394759.92, 34602...",0.066289,Articulado (23m),False
1,3117,76,SAPOPEMBA,SAP,"MULTILINESTRING ((347423.675 7389607.4, 347253...",0.223807,Básico,False
2,3117,73,SAO MATEUS,SMT,"MULTILINESTRING ((350506.219 7387400.996, 3474...",10.452956,Básico,False


In [13]:
#acho que não precisa fazer isso agora agora, pode ser depois
gdf_overlay = gdf_overlay.merge(
    consumo_diesel, 
    left_on="modelo", 
    right_on="tecnologia", 
    how="left"
)

## aleatorização do ar condicionado
np.random.seed(42)
gdf_overlay["tem_ar"] = np.random.choice([True, False], size=len(gdf_overlay))

In [14]:
gdf_overlay['consumo_l_diesel'] = gdf_overlay['distancia'] * np.where(
    gdf_overlay['tem_ar'], #bool
    gdf_overlay['com_ar_l_km'], #valor se True
    gdf_overlay['sem_ar_l_km'] #valor se False
)

gdf_overlay['consumo_kg_diesel'] = gdf_overlay['distancia'] * np.where(
    gdf_overlay['tem_ar'], #bool
    gdf_overlay['com_ar_kg_km'], #valor se True
    gdf_overlay['sem_ar_kg_km'] #valor se False
)

In [15]:
gdf_overlay['emissao_co2'] = gdf_overlay['consumo_l_diesel'] * 2.671 #fator de emissão de CO2 do diesel

In [16]:
gdf_overlay.loc[gdf_overlay['sg_distrit']=='TAT']

,id_onibus,cd_distrit,nm_distrit,sg_distrit,geometry,distancia,modelo,is_eletrico,tecnologia,sem_ar_l_km,sem_ar_kg_km,com_ar_l_km,com_ar_kg_km,tem_ar,consumo_l_diesel,consumo_kg_diesel,emissao_co2
3,3117,80,TATUAPE,TAT,"LINESTRING (341149.206 7394098.499, 339982.012...",2.929080,Básico,False,Básico,0.46,0.386,0.53,0.445,True,1.552412,1.303441,4.146493
1751,21333,80,TATUAPE,TAT,"LINESTRING (340084.121 7397219.091, 340266.003...",1.010833,Miniônibus,False,Miniônibus,0.30,0.252,0.35,0.294,False,0.303250,0.254730,0.809980
2135,21933,80,TATUAPE,TAT,"MULTILINESTRING ((338208.076 7397107.777, 3382...",2.022147,Articulado (18m),False,Articulado (18m),0.71,0.596,0.80,0.672,True,1.617718,1.358883,4.320925
2392,22494,80,TATUAPE,TAT,"LINESTRING (339501.702 7396272.573, 339729.984...",1.388746,Básico,False,Básico,0.46,0.386,0.53,0.445,False,0.638823,0.536056,1.706297
2713,22776,80,TATUAPE,TAT,"LINESTRING (338947.529 7396362.148, 339092.185...",1.939017,Miniônibus,False,Miniônibus,0.30,0.252,0.35,0.294,False,0.581705,0.488632,1.553734
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8537,55476,80,TATUAPE,TAT,"LINESTRING (340194.432 7396457.624, 340125.941...",1.384012,Miniônibus,False,Miniônibus,0.30,0.252,0.35,0.294,True,0.484404,0.406899,1.293843
8555,55497,80,TATUAPE,TAT,"LINESTRING (339392.354 7394812.584, 339404.231...",0.972754,Padron,False,Padron,0.55,0.462,0.63,0.529,True,0.612835,0.514587,1.636882
8562,55506,80,TATUAPE,TAT,"LINESTRING (340197.745 7394877.398, 340006.102...",1.331039,Básico,False,Básico,0.46,0.386,0.53,0.445,True,0.705450,0.592312,1.884258
8665,56323,80,TATUAPE,TAT,"LINESTRING (341350.855 7396988.088, 341291.907...",1.281357,Miniônibus,False,Miniônibus,0.30,0.252,0.35,0.294,False,0.384407,0.322902,1.026752


In [17]:
gdf_overlay.groupby('nm_distrit').agg({
    'distancia':'sum',
    "emissao_co2": 'sum',
    'is_eletrico': 'first'
}).reset_index()

gdf_overlay

,id_onibus,cd_distrit,nm_distrit,sg_distrit,geometry,distancia,modelo,is_eletrico,tecnologia,sem_ar_l_km,sem_ar_kg_km,com_ar_l_km,com_ar_kg_km,tem_ar,consumo_l_diesel,consumo_kg_diesel,emissao_co2
0,3106,24,CIDADE LIDER,CLD,"MULTILINESTRING ((345990.634 7394759.92, 34602...",0.066289,Articulado (23m),False,Articulado (23m),0.75,0.630,0.85,0.714,True,0.056346,0.047331,0.150500
1,3117,76,SAPOPEMBA,SAP,"MULTILINESTRING ((347423.675 7389607.4, 347253...",0.223807,Básico,False,Básico,0.46,0.386,0.53,0.445,False,0.102951,0.086389,0.274982
2,3117,73,SAO MATEUS,SMT,"MULTILINESTRING ((350506.219 7387400.996, 3474...",10.452956,Básico,False,Básico,0.46,0.386,0.53,0.445,True,5.540067,4.651565,14.797518
3,3117,80,TATUAPE,TAT,"LINESTRING (341149.206 7394098.499, 339982.012...",2.929080,Básico,False,Básico,0.46,0.386,0.53,0.445,True,1.552412,1.303441,4.146493
4,3117,85,VILA FORMOSA,VFO,"MULTILINESTRING ((343631.382 7392321.823, 3411...",6.043577,Básico,False,Básico,0.46,0.386,0.53,0.445,True,3.203096,2.689392,8.555468
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14162,90999,4,ARICANDUVA,ARI,"LINESTRING (344805.977 7393908.946, 344173.133...",2.182197,Articulado (18m),False,Articulado (18m),0.71,0.596,0.80,0.672,False,1.549360,1.300590,4.138341
14163,90999,59,PENHA,PEN,"MULTILINESTRING ((347591.953 7397655.787, 3477...",2.810272,Articulado (18m),False,Articulado (18m),0.71,0.596,0.80,0.672,False,1.995293,1.674922,5.329427
14164,90999,91,VILA MATILDE,VMT,"LINESTRING (345830.414 7397289.64, 344983.38 7...",2.920772,Articulado (18m),False,Articulado (18m),0.71,0.596,0.80,0.672,True,2.336618,1.962759,6.241106
14165,90999,5,ARTUR ALVIM,AAL,"MULTILINESTRING ((347727.842 7396698.937, 3480...",4.654722,Articulado (18m),False,Articulado (18m),0.71,0.596,0.80,0.672,True,3.723778,3.127973,9.946211


In [18]:
save_shp(gdf_overlay, "gdf_overlay.shp")

c:\Users\x050258\projetos\dash_onibus_eletricos\utils\save_shp.py:19: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file(file_path)
c:\Users\x050258\projetos\dash_onibus_eletricos\venv\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'is_eletrico' to 'is_eletric'
  ogr_write(
c:\Users\x050258\projetos\dash_onibus_eletricos\venv\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'sem_ar_l_km' to 'sem_ar_l_k'
  ogr_write(
c:\Users\x050258\projetos\dash_onibus_eletricos\venv\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'sem_ar_kg_km' to 'sem_ar_kg_'
  ogr_write(
c:\Users\x050258\projetos\dash_onibus_eletricos\venv\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'com_ar_l_km' to 'com_ar_l_k'
  ogr_write(
c:\Users\x050258\projetos\dash_onibus_eletricos\venv\Lib\site-packages\p

Shapefile salvo em data\gdf_overlay.shp
